## Коэффициенты корреляции

**1.** Есть ли связь между неграмотностью и рождаемостью? Для 94 стран, уровень неграмотности женщин в которых больше 5%, известны доля неграмотных среди женщин старше 15 (на 2003 год) и средняя рождаемость на одну женщину (на 2005 год).

Чему равен выборочный коэффициент корреляции Пирсона между этими двумя признаками? Округлите до четырёх знаков после десятичной точки. 

In [1]:
import pandas as pd
import numpy as np
from scipy import stats

In [2]:
data = pd.read_csv("data/illiteracy.txt", sep="\t", header=0)
data.head()

,Country,Illit,Births
0,Albania,20.5,1.78
1,Algeria,39.1,2.44
2,Bahrain,15.0,2.34
3,Belize,5.9,2.97
4,Benin,73.5,5.60


In [3]:
pirs_coef = np.sum(
    (data["Illit"] - data["Illit"].mean()) * (data["Births"] - data["Births"].mean()))/ \
    np.sqrt(np.sum((data["Illit"] - data["Illit"].mean())** 2)  * np.sum((data["Births"] - data["Births"].mean())** 2) )

In [4]:
assert round(pirs_coef, 4) == round(np.corrcoef(data["Illit"], data["Births"])[0][1], 4)
print(round(pirs_coef, 4))

0.7687


**2.** Чему равен выборочный коэффициент корреляции Спирмена признаков из предыдущего вопроса? Округлите до четырёх знаков после десятичной точки.

In [5]:
round(stats.spearmanr(data["Illit"], data["Births"])[0], 4)

0.753

## Корреляционный анализ

**1.** Для 61 большого города в Англии и Уэльсе известны средняя годовая смертность на 100000 населения (по данным 1958–1964) и концентрация кальция в питьевой воде (в частях на маллион). Чем выше концентрация кальция, тем жёстче вода. Города дополнительно поделены на северные и южные.

Есть ли связь между жёсткостью воды и средней годовой смертностью? Посчитайте значение коэффициента корреляции Пирсона между этими признаками, округлите его до четырёх знаков после десятичной точки.

In [6]:
water_data = pd.read_csv("data/water.txt", sep="\t", header=0)
water_data.head()

,location,town,mortality,hardness
0,South,Bath,1247,105
1,North,Birkenhead,1668,17
2,South,Birmingham,1466,5
3,North,Blackburn,1800,14
4,North,Blackpool,1609,18


In [7]:
round(np.corrcoef(water_data["hardness"], water_data["mortality"])[0][1], 4)

-0.6548

**2.** В предыдущей задаче посчитайте значение коэффициента корреляции Спирмена между средней годовой смертностью и жёсткостью воды. Округлите до четырёх знаков после десятичной точки. 

In [8]:
round(stats.spearmanr(water_data["hardness"], water_data["mortality"])[0], 4)

-0.6317

**3.** Сохраняется ли связь между признаками, если разбить выборку на северные и южные города? Посчитайте значения корреляции Пирсона между средней годовой смертностью и жёсткостью воды в каждой из двух подвыборок, введите наименьшее по модулю из двух значений, округлив его до четырёх знаков после десятичной точки.

In [9]:
round(
    np.corrcoef(water_data[water_data.loc[:, 'location'].values =='South']["hardness"],
                water_data[water_data.loc[:, 'location'].values =='South']["mortality"])[0][1], 4)

-0.6022

In [10]:
round(
    np.corrcoef(water_data[water_data.loc[:, 'location'].values =='North']["hardness"],
                water_data[water_data.loc[:, 'location'].values =='North']["mortality"])[0][1], 4)

-0.3686

**4.** Среди респондентов General Social Survey 2014 года хотя бы раз в месяц проводят вечер в баре 203 женщины и 239 мужчин; реже, чем раз в месяц, это делают 718 женщин и 515 мужчин.  

Посчитайте значение коэффициента корреляции Мэтьюса между полом и частотой похода в бары. Округлите значение до трёх знаков после десятичной точки.

In [11]:
def Matthews_correlation(a, b, c, d):
    return (a*d-b*c)/np.sqrt((a+b)*(a+c)*(b+d)*(c+d))

men_visit_monthly = 239
women_visit_monthly = 203
men_visit_rarely = 515
women_visit_rarely = 718
print(f'Matthews correlation is: {np.round(Matthews_correlation(men_visit_monthly, men_visit_rarely, women_visit_monthly, women_visit_rarely), 3)}')

Matthews correlation is: 0.109


**5.** В предыдущей задаче проверьте, значимо ли коэффициент корреляции Мэтьюса отличается от нуля. Посчитайте достигаемый уровень значимости; используйте функцию scipy.stats.chi2_contingency. Введите номер первой значащей цифры (например, если вы получили $5.5\times10^{-8}$, нужно ввести 8).

In [12]:
chi_stat = stats.chi2_contingency([[men_visit_monthly, men_visit_rarely], [women_visit_monthly, women_visit_rarely]])
print("Chi2 p-value is: %f" % chi_stat[1])

Chi2 p-value is: 0.000011


**6.** В предыдущей задаче давайте попробуем ответить на немного другой вопрос: отличаются ли доля мужчин и доля женщин, относительно часто проводящих вечера в баре? Постройте 95% доверительный интервал для разности долей, вычитая долю женщин из доли мужчин. Чему равна его нижняя граница? Округлите до четырёх знаков после десятичной точки.

In [13]:
def proportions_confint_diff_ind(p1, count1, p2, count2, alpha = 0.05):    
    z = stats.norm.ppf(1 - alpha / 2.)
    
    left_boundary = (p1 - p2) - z * np.sqrt(p1 * (1. - p1)/ count1 + p2 * (1 - p2)/ count2)
    right_boundary = (p1 - p2) + z * np.sqrt(p1 * (1. - p1)/ count1 + p2 * (1 - p2)/ count2)
    
    return (left_boundary, right_boundary)

all_men = men_visit_monthly + men_visit_rarely
all_women = women_visit_monthly + women_visit_rarely
men_proportion = float(men_visit_monthly)/float(all_men)
women_proportion = float(women_visit_monthly)/float(all_women)
conf_interval_diff = proportions_confint_diff_ind(men_proportion, all_men, women_proportion, all_women)
print("Interval for men and women bar visitors proportion diff: [%.4f, %.4f]" % (np.round(conf_interval_diff[0],4),np.round(conf_interval_diff[1],4)))

Interval for men and women bar visitors proportion diff: [0.0539, 0.1392]


In [14]:
def proportions_diff_z_stat_ind(p1, count1, p2, count2, alpha = 0.05):
    P = float(p1*count1 + p2*count2) / (count1 + count2)
    
    return (p1 - p2) / np.sqrt(P * (1 - P) * (1. / count1 + 1. / count2))

def proportions_diff_z_test(z_stat, alternative = 'two-sided'):
    if alternative not in ('two-sided', 'less', 'greater'):
        raise ValueError("alternative not recognized\n"
                         "should be 'two-sided', 'less' or 'greater'")
    
    if alternative == 'two-sided':
        return 2 * (1 - stats.norm.cdf(np.abs(z_stat)))
    
    if alternative == 'less':
        return stats.norm.cdf(z_stat)

    if alternative == 'greater':
        return 1 - stats.norm.cdf(z_stat)


pvalue = proportions_diff_z_test(proportions_diff_z_stat_ind(men_proportion, all_men, women_proportion, all_women))
print(f'p-value for men-women proportion difference is: {pvalue}')

p-value for men-women proportion difference is: 8.153453089576601e-06


In [15]:
contingency_table = np.array([[197, 111, 33], [382, 685, 331], [110, 342, 333]])
chi_stat_happiness = stats.chi2_contingency(contingency_table)
print(f'Chi2 statistic value is: {np.round(chi_stat_happiness[0], 4)}')

Chi2 statistic value is: 293.6831


In [16]:
print(f"Chi2 p-value is: {chi_stat_happiness[1]}")

Chi2 p-value is: 2.4964299580093467e-62


In [17]:
def v_Cramer_correlation(table):
    chi_stat = stats.chi2_contingency(table)[0]
    k_min = np.min(table.shape)
    n = np.sum(table)
    return np.sqrt(chi_stat/(n*(k_min-1)))

print(f'V-Cramer statistic is: {np.round(v_Cramer_correlation(contingency_table), 4)}')

V-Cramer statistic is: 0.2412
